## Minería de Datos (Master en Data Science, UIMP-UC) 
## Tarea 2. Problemas de Clasificación - Reducción de la Dimensión
### [Profesores: Steven Van Vaerenbergh, Rodrigo G. Manzanas, Joaquín Bedia y Sixto Herrera]

### Estudiante: Nicolò Trevisani

En la presente tarea consideraremos el dataset `meteo.csv`, que podéis descargaros en el GitHub dedicado a este Máster ([meteo.csv](https://github.com/SantanderMetGroup/Master-Data-Science/blob/master/Data_mining/datasets/meteo.csv.)) y que ha sido utilizado en diferentes sesiones prácticas. Dicho dataset contiene en la primera columna el valor de precipitación observado en Lisboa en el periodo 1979-2008 mientras que en las restantes contiene los valores observados de diferentes variables atmosféricas en 40 puntos que cubren aproximadamente la Península Ibérica. Dichas variables serán los `predictores` del modelo mientras que la precipitación será nuestra variable objetivo. En particular los predictores son:

* Altura geopotencial en 500 hPa (columnas 2:41),
* Temperatura del aire en 850 hPa (columnas 42:81), 700 hPa (columnas 82:121) y 500 hPa (columnas 122:161), 
* Temperatura del aire en superficie (columnas 162:201),
* Humedad específica del aire en 850 hPa (columnas 202:241) y 500 hPa (columnas 242:281) y 
* Presión al nivel del mar (columnas 282:321)

Para establecer el problema de clasificación consideraremos dos umbrales de discretización, 1 mm y 20 mm, que definen la ocurrencia de precipitación (Wet days) y de precipitaciones intensas (Very heavy precipitation days), respectivamente. Puedes consultar más detalles de la definición en la web de [ECA&D](https://www.ecad.eu//indicesextremes/indicesdictionary.php).

Para el desarrollo de la tarea se permitirá el uso de todo el material incluido en el Moodle de las asignatura así como el desarrollado por el alumno durante la realización de las prácticas.

La entrega consisitirá de un notebook de Jupyter ó un R-MarkDown, junto con el archivo html que éste genera. Ambos ficheros se entregarán a través del Moodle de la asignatura en la tarea correspondiente.

### Punto 1 (3 puntos):

En esta primera parte de la tarea trataremos de ilustrar parte de los problemas reflejados en las sesiones teórico-prácticas planteando diferentes experimentos con el dataset `meteo.csv`.
En primer lugar, considerar el dataset completo incluyendo tanto las variables predictoras como la variable objetivo. Por un lado, calculad las componentes principales con y sin estandarización (`Nota:` consultar la ayuda de la función scale -> `? scale`):

* ¿Cómo contribuye en cada caso la variable objetivo a la primera componente principal?
* ¿A qué componente principal contribuye principalmente la variable objetivo? ¿qué porcentaje de varianza se explica hasta dicha componente?
* En base a los puntos anteriores, ¿puede considerarse en alguno de los casos que la variable objetivo se "eliminaría" del modelo debido a su contribución a cada una de las componentes principales?
* ¿Cómo se distribuye la contribución de las diferentes variables del espacio original a la primera componente principal? En caso de no estandarizar, ¿puede inferirse alguna relación entre el rango de las variables y su contribución a la primera componente principal?

`Nota 1:` en principio es posible trabajar con el dataset completo sin seleccionar un subconjunto pero si surge algún problema de memoria podéis considerar únicamente los primeros 10 años (~3650 filas).

`Nota 2:` usad las herramientas gráficas vistas durante la práctica para ilustrar los resultados y las conclusiones obtenidas.

## Desarrollo Punto 1

In [11]:
meteo <- read.csv("meteo.csv", row.name = 1)

In [12]:
head(meteo)

,y,X1,X2,X3,X4,X5,X6,X7,X8,X9,⋯,X311,X312,X313,X314,X315,X316,X317,X318,X319,X320
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,10.9,57042.96,56534.84,55884.40,55175.59,54458.21,56980.27,56460.59,55800.02,55051.15,⋯,101985.9,101627.3,101357.2,101110.7,101291.0,101814.9,101479.4,101164.5,100970.7,100905.3
2,0.6,56963.20,56493.45,55930.82,55340.20,54768.63,56856.57,56361.95,55776.76,55164.26,⋯,102048.1,102069.7,102204.5,102480.3,102858.8,101973.3,102033.3,102165.5,102323.9,102534.2
3,13.0,56522.59,55971.40,55304.34,54498.03,53725.15,56514.84,55977.71,55327.28,54569.78,⋯,102399.1,102448.4,102438.3,102434.3,102339.8,102669.7,102565.5,102530.3,102453.6,102487.1
4,0.0,54627.98,53979.80,53494.05,53072.98,52616.36,54776.17,54089.05,53516.98,53012.86,⋯,100970.0,100672.8,100360.7,100133.7,100028.4,101126.2,100700.0,100381.3,100118.3,100319.8
5,0.0,53584.38,53391.01,53310.38,53292.82,53340.13,53781.70,53498.13,53306.82,53213.26,⋯,100652.7,100663.5,100747.7,100934.0,101015.8,100801.6,100685.3,100777.5,100909.4,101164.7
6,1.2,54221.86,54220.24,54328.55,54772.49,55251.11,54391.11,54345.93,54284.05,54525.99,⋯,101358.6,101511.5,101798.9,102142.2,102368.5,101438.7,101440.9,101685.3,102030.7,102311.9


Cargo las librerias necesarias para hacer una *principal component analysis*

In [52]:
# Load the libraries for PCA
library("FactoMineR")
library("factoextra")

### Saco las componentes principales, sin escalar las variables e incluiendo la variable objetivo

In [23]:
meteo.pca <- PCA(meteo,  graph = FALSE)

In [24]:
get_eig(meteo.pca)

,eigenvalue,variance.percent,cumulative.variance.percent
Dim.1,"186,3477415","58,0522559","58,05226"
Dim.2,"50,8560817","15,8430161","73,89527"
Dim.3,"12,5584062","3,9122761","77,80755"
Dim.4,"8,9703826","2,7945117","80,60206"
Dim.5,"7,7530408","2,4152775","83,01734"
Dim.6,"7,0236376","2,1880491","85,20539"
Dim.7,"5,4831912","1,7081592","86,91355"
Dim.8,"4,6532328","1,4496052","88,36315"
Dim.9,"3,2313884","1,0066630","89,36981"
Dim.10,"2,7745512","0,8643462","90,23416"


In [51]:
meteo.var <- get_pca_var(meteo.pca)
head(meteo.var$contrib)

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
y,"0,01068821","0,1481743","0,409296737","0,05626072","0,01623840"
X1,"0,32967378","0,3751500","0,003521151","1,08009249","0,18075169"
X2,"0,30982013","0,5400883","0,013331685","1,09103897","0,03021082"
X3,"0,28721888","0,6817707","0,037136982","0,90123571","0,02682396"
X4,"0,25893709","0,7623370","0,083838768","0,59147460","0,29705941"
X5,"0,22439603","0,7564979","0,158458134","0,29112473","0,84317813"


In [50]:
print(paste("La constribución de la variable objetivo a la primera componente principal es:",
            format(round(meteo.var$contrib[1,1]*100,4), nsmall = 4),"%"))

[1] "La constribución de la variable objetivo a la primera componente principal es: 1,0688 %"


In [16]:
?scale

### Punto 2 (3 puntos):

A continuación, consideraremos la serie de precipitación discretizada a partir del valor 1 mm, que es el estándar definido para establecer los días en que ha llovido. El objetivo de este apartado es predecir la ocurrencia de precipitación en Lisboa a partir de los predictores antes definidos. Para ello consideraremos, por un lado, las componentes principales obtenidas estandarizando los datos originales y, por otro lado, el método `KNN`. Dividir la muestra en dos subconjuntos, el primero (20 primeros años) lo utilizaremos para calibrar el modelo y obtener su configuración óptima, mientras que el segundo (10 últimos años) lo utilizaremos como test independiente de cara a comparar diferentes métodos.

* Considerad diferentes umbrales de varianza explicada y el número de PCs asociado (p.e. 40%, 60%, 80% y 90%) y obtend el valor óptimo de `K` en cada caso rastreando valores entre 1 y 15, ¿cómo cambia el valor óptimo? En base a los resultados obtenidos, ¿cuantas PCs considerarías para entrenar el modelo? (`Nota:` ver práctica de `KNN`)
* Para la configuración óptima calibrada con el conjunto de entrenamiento realizad la predicción sobre el conjunto de test y estimad los errores cometidos sobre este conjunto.

### Punto 3 (2 puntos):

Repetid el experimento anterior considerando la precipitación discretizada a partir del valor 20 mm, que es el estándar definido para establecer los días en que ha llovido de forma intensa.

* ¿Cómo cambian los valores óptimos de `K` y de número de PCs? 
* ¿Cómo cambian los errores sobre el conjunto de test en este caso?
* En caso de existir diferentes significativas en ambos casos, ¿a qué crees que puede ser debido? ¿La frecuencia en la muestra del evento a predecir puede influir en la calidad de la predicción?

### Punto 4 (2 puntos):

Considerando el número de PCs óptimo obtenido en el apartado anterior, predecid el conjunto de test utilizando en este caso la regresión logística como método de clasificación (`Nota:` ver `Práctica Clasificacion Lineal` en el Moodle de la asignatura de Estadística o en la de Minería de Datos). 

* Comparad los resultados obtenidos utilizando ambos métodos para la predicción de la ocurrencia de precipitación y de precipitación intensa (`Nota:` considerar, por ejemplo, la curva ROC para la comparación), ¿alguno de los métodos se comporta mejor que el otro de forma sistemática?